# Semianr 5 - Applied Quantitative Logistics

In [ ]:
import math
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Hub Location Allocation

In [ ]:
N = 40    # Number of customers
M = 20    # Number of centers

# Generate the x coordinate of customer
xc = list(np.random.randint(0, 100, N))
print(xc)

In [ ]:
# Generate the y coordinate of customer
yc = list(np.random.randint(0, 100, N))
print(yc)

In [ ]:
# Generate demand for each customer
d = list(np.random.randint(5, 50, N))
print(d)

In [ ]:
# Generate the x coordinate of centers
xs = list(np.random.randint(0, 99, M))
print(xs)

In [ ]:
# Generate the y coordinate of centers
ys = list(np.random.randint(0, 99, M))
print(ys)

In [ ]:
def hubLocation():
    
    # Customer information
    xc = [94, 60, 62, 3, 23, 43, 27, 62, 97, 75, 54,
          56, 58, 0, 62, 28, 27, 86, 7, 52, 65, 0, 32,
          15, 3, 61, 16, 65, 63, 43, 67, 53, 98, 68,
          34, 32, 26, 18, 33, 66]
    
    yc = [67, 33, 11, 2, 48, 88, 17, 71, 38, 78,
          37, 72, 58, 12, 54, 5, 24, 76, 38, 61,
          82, 65, 66, 55, 17, 79, 72, 14, 45, 30,
          36, 97, 15, 52, 30, 66, 13, 42, 77, 12]
    
    d = [13, 44, 12, 30, 5, 44, 18, 29, 20, 6, 35,
         16, 17, 31, 32, 9, 37, 30, 8, 21, 31, 17,
         47, 11, 6, 14, 40, 48, 31, 16, 32, 14, 44,
         42, 6, 26, 20, 43, 14, 20]
    
    N = len(xc)
    
    # Service center information
    xs = [63, 91, 0, 54, 39, 55, 15, 50, 21, 92,
          12, 81, 41, 5, 62, 75, 62, 20, 77, 57]
    
    ys = [37, 25, 37, 12, 43, 57, 98, 2, 85, 87,
          0, 75, 30, 30, 85, 14, 65, 78, 95, 36]
    
    M = len(xs)
    
    # calculate the distance
    D = np.zeros([N, M])
    
    for i in range(N):
        for j in range(M):
            D[i][j] = abs(xc[i]-xs[j]) + abs(yc[i]-ys[j])
    
    model = {'N': N,
             'M': M,
             'xc': xc,
             'yc': yc,
             'xs': xs,
             'ys': ys,
             'd': d,
             'D': D}
    
    return model

In [ ]:
model = hubLocation()
model['D'].shape

In [ ]:
model['ys'][4]

### Create Random Solution

In [ ]:
def createRandomSolution(model):
    M = model['M']
    f = list(np.random.randint(0, 2, M))
    
    return f

In [ ]:
createRandomSolution(model)

### Cost Function

In [ ]:
def myCost(f, model):
    
    global NFE
    
    if pd.isna(NFE):
        NFE = 0

    NFE += 1
    
    # If no center is activated
    # The cost of the system is inf
    if (np.all(np.array(f) == 0)):
        z = math.inf
        return z

    N = model['N']
    M = model['M']
    D = model['D']
    
    D_min = np.zeros(N)
    
    for i in range(N):
        D_temp = []
        for j in range(M):
            if f[j] == 1:
                D_temp.append(D[i][j])
                
        D_min[i] = min(D_temp)
        
    z = sum(np.array(model['d']) * np.array(D_min))
    
    return z

### Sorting Population

In [ ]:
# Sort the population and cost (based on the cost)
def pop_sort(p, c):
    li = []
    for i in range(len(c)):
        li.append([c[i],i])
        
    li.sort()
    sort_index = []
    
    for x in li:
        sort_index.append(x[1])
    
    positions, cost = [], []
    for i in sort_index:
        positions.append(p[i])
        cost.append(c[i])
        
    return positions, cost

### Roullete Wheel Selection

In [ ]:
def rouletteWheelSelection(p):
    r = random.random()
    
    c = np.cumsum(p)
    
    indexes = [
    index for index in range(len(c))
    if c[index] > r
    ]
    
    return indexes[0]

### Crossover

In [ ]:
# Uniform Crossover is better than double point crossover better than single point crossover

# Single point crossover
def singlePoint_crossover(x1, x2):
    index = int(np.random.randint(1, len(x1)-1, size=1))
    
    y1 = x1[:index] + x2[index:]
    y2 = x2[:index] + x1[index:]
    
    return y1, y2

# Double Point Crossover
def doublePoint_crossover(x1, x2):
    ind = random.sample(range(1, len(x1)-1), 2)
    
    index1 = min(ind)
    index2 = max(ind)
    
    # Another way is to generate sequence from, 1 to len(x1)-1 then shuffle it
    # Then select first two elements 
    # (it won't be the same at all) --> my_ind = list(range(1, len(x1)-1))
    # random.shuffle(my_list)
    y1 = x1[:index1] + x2[index1:index2] + x1[index2:]
    y2 = x2[:index1] + x1[index1:index2] + x2[index2:]
    
    return y1, y2

# Uniform Crossover
def uniform_crossover(x1, x2):
    alpha = list(np.random.randint(2, size=len(x1)))
    
    y1 = list(np.multiply(alpha, x1) + (1-np.array(alpha)) * np.array(x2))
    y2 = list(np.multiply(alpha, x2) + (1-np.array(alpha)) * np.array(x1))
    
    return y1, y2

def CrossOver(x1, x2):
    
    pSinglePoint = 0.1
    pDoublePoint = 0.2
    pUniform = 1-pSinglePoint-pDoublePoint
    
    METHOD = rouletteWheelSelection([pSinglePoint, pDoublePoint, pUniform])
    
    if METHOD == 0:
        y1, y2 = singlePoint_crossover(x1, x2)
    elif METHOD == 1:
        y1, y2 = doublePoint_crossover(x1, x2)
    elif METHOD == 2:
        y1, y2 = uniform_crossover(x1, x2)
    
    return y1, y2

### Mutation

In [ ]:
def Mutation(x):
    index = int(np.random.randint(0, len(x), size=1))
    
    y = x.copy()
    
    y[index] = 1-x[index]
    
    return y

### GA Algorithm

In [ ]:
### Problem Parameters Definition ###

model = hubLocation()

nVar = model['M']       # Number of decision variables

global NFE
NFE = 0

### GA Parameters ###
maxIt = 75     # Maximum numner of iterations
nPop = 20       # Population size 

pc = 0.8                   # Crossover percentage
nc = 2*round(pc*nPop/2)    # Number of offsprings (parents)

pm = 0.3                   # Mutation percentage
nm = round(pm*nPop)        # Number of mutants2 = unifrnd(0,2 = unifrnd(0,

### Initialization ###
pop, costs = [], []

for i in range(0, nPop):
    pop.append(createRandomSolution(model))
    costs.append(myCost(pop[i], model))

# Sort the population and costs
pop, costs = pop_sort(pop, costs)

#  Store the best solution
bestSolution = [pop[0]]

# Array to hold best cost values
bestCosts = [costs[0]]

# Store the NFE into the array
nfe = [NFE]

### Main Loop ###
for it in range(1, maxIt):
    
    # Crossover
    popc, popc_cost = [], []
    for k in range(1, int(nc/2)):
        
        # Select parent indices
        rand1 = int(np.random.randint(nPop, size=1))
        rand2 = int(np.random.randint(nPop, size=1))
        
        # Select parents
        p1 = pop[rand1]
        p2 = pop[rand2]
        
        # Apply crossover
#         y1, y2 = singlePoint_crossover(p1, p2)
#         y1, y2 = doublePoint_crossover(p1, p2)
#         y1, y2 = uniform_crossover(p1, p2)

        y1, y2 = CrossOver(p1, p2)
        
        # Store the offspring after crossover
        popc.append(y1)
        popc.append(y2)
        
        # Evaluate the offspring
        popc_cost.append(myCost(y1, model))
        popc_cost.append(myCost(y2, model))
        
    # Mutation
    popm, popm_cost = [], []
    for k in range(1, nm):
        
        # Select parent
        rand = int(np.random.randint(nPop, size=1))
        p = pop[rand]
        
        # Apply Mutation
        popm.append(Mutation(p))
        
        # Evaluate the offspring
        popm_cost.append(myCost(popm[-1], model))
        
    # Create merged population
    pop = pop + popm + popc
    costs = costs + popm_cost + popc_cost
    
    # sort the whole population
    pop, costs = pop_sort(pop, costs)
    
    # Truncation
    pop = pop[:nPop]
    costs = costs[:nPop]
    
    # Store the best solution
    bestSolution.append(pop[0])
    
    # Store the best cost
    bestCosts.append(costs[0])
    
    # Append NFE to the array
    nfe.append(NFE)
    
#     if bestCosts[-2] == 0:
#         break
        
    print(f'Iteration {it} : NFE = {nfe[-1]}, Best Cost = {bestCosts[it]}')

### Results

In [ ]:
# Plot the result
plt.plot(bestCosts, linewidth = 3)
plt.xlabel('NFE')
plt.ylabel('Costs')